# Introduction

One will be implementing a basic NLP model to classify phrases within an API, using some of the MLOPs techniques.

In [21]:
#! pip install textblob
from textblob import TextBlob

In [22]:
phrase = 'Python is a good Machine learning language'
tb = TextBlob(phrase)
tb.sentiment.polarity

0.7

# A simple APi code

In [23]:
from flask import Flask

app = Flask(__name__)

@app.route('/')
def home():
    return 'My first API.'

#app.run(debug=True,
#  use_reloader=False) #This will allow one to use the API directly form the notebook, without a main.py file, for example


# Adding new endpoints

In [24]:
from flask import request

app = Flask(__name__)

@app.route('/')
def home():
    return 'My first API.'

# Analyzing the polarity of certain phrase
@app.route('/sentiment/<phrase>')

def sentiment(phrase):
    tb = TextBlob(phrase)
    polarity = tb.sentiment.polarity
    return "Polarity: {}".format(polarity)

#app.run(debug=True,use_reloader=False)

# Modelling based on user input

One will be using a very basic model, will not optimize or do feature engineering. This is only to show how to deploy a model into an API.

In [25]:
import pandas as pd

In [26]:
house_data = pd.read_csv('casas.csv')

x = house_data.drop('preco', axis=1)
y= house_data['preco']

from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)

In [27]:
features = x.columns.to_list()

In [28]:
from sklearn.linear_model import LinearRegression
model = LinearRegression()
model.fit(x_train, y_train)

LinearRegression()

In [29]:
model.predict(
    [[120, #tamanho
     2022, #ano
     2, # garagem
    ]])

# this will be executed inside the API using the code bellow

C:\Users\Gustavo Fortunato\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


array([207440.42632476])

In [30]:
@app.route('/pricing/<int:tamanho>/<int:ano>/<int:garagem>')
def pricing(tamanho, ano, garagem):
    price = f" A house with a size of {tamanho}, built in the year of {ano} and with {garagem} garage size has the price of {model.predict([[tamanho, ano, garagem]])[0]:.2f}"
    return price

#app.run(debug=True,use_reloader=False)

This code considers that the dataset loading and model training will be executed before the API initializes, but in the same code, just after the imports. Like the example bellow.

In [31]:
from flask import Flask, request
from sklearn.linear_model import LinearRegression
import pandas as pd

#machine learning
from sklearn.model_selection import train_test_split

#This is how to implement a ML model, but this could break down the API 
#if the model is too complex or the dataset is too big.

#loading the dataset
house_data = pd.read_csv('casas.csv')

x = house_data.drop('preco', axis=1)
features = x.columns.to_list()
y= house_data['preco']

#spliting the data
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)

#Instancing and training the model

model = LinearRegression()
model.fit(x_train, y_train)

@app.route('/pricing/<int:tamanho>/<int:ano>/<int:garagem>')
def pricing2(tamanho, ano, garagem):
    price = f" A house with a size of {tamanho}, built in the year of {ano} and with {garagem} garage size has the price of {model.predict([[tamanho, ano, garagem]])[0]:.2f}"
    return price

#app.run(debug=True, use_reloader=False) 

# The problem of ML pipeline inside the API

One is facing a problem that could break down its API. Whenever one inicializes the API the process of loading the data and training the model happens. Using a small dataset and the model one was using, this problem does not exist, but when scaling happens, this problem breaks the API. 

One way to fix that problem is to export the model already trained and only starting the model, without the process of readind the data, instancing and training the model.

In [32]:
#for the api one needs an object with the information

json_of_features = {
    "tamanho" : 120,
    "ano" : 2022,
    "garagem" : 2,
}

In [33]:
# this will allow one to use the model trained file
import pickle

pickle.dump(model, open('model.sav', 'wb'))

In [34]:
model = pickle.load(open('model.sav', 'rb'))
features =['tamanho', 'ano', 'garagem']

app = Flask(__name__)

@app.route('/')
def home():
    return 'My first API.'

@app.route('/sentiment/<phrase>')
def sentiment(phrase):
    tb = TextBlob(phrase)
    polarity = tb.sentiment.polarity
    return "Polarity: {}".format(polarity)

@app.route('/pricing/', methods=['POST'])
def pricing():
    data = request.get_json()
    input_data = [data[col] for col in features]
    price = f" A house with a size of {input_data[0]}, built in the year of {input_data[1]} and with {input_data[2]} garage size has the price of {model.predict([input_data])[0]:.2f}"
    return price

#app.run(debug=True, use_reloader=False)

To execute the requests, one will need the above code to be ran outsite the notebook, since it requires a localhost, and the notebbok can't execute the localhost and a request at the same time.

In [35]:
import requests
url = 'http://127.0.0.1:5000/pricing/'
response= requests.post(url, json=json_of_features)
response.text

''

# Adding autentication

As a minimal security level one will need a login and password to execute any function of the api., this will result on a new version of the api.

In [36]:
from flask_basicauth import BasicAuth
app = Flask(__name__)

#basic authentication
app.config['BASIC_AUTH_USERNAME'] = 'someone'
app.config['BASIC_AUTH_PASSWORD'] = 'password'
auth = BasicAuth(app)

#model
model = pickle.load(open('model.sav', 'rb'))
features =['tamanho', 'ano', 'garagem']

#endpoints - home
@app.route('/')
def home():
    return 'My first API.'

#endpoints - sentiment analysys
@app.route('/sentiment/<phrase>')
@auth.required
def sentiment(phrase):
    tb = TextBlob(phrase)
    polarity = tb.sentiment.polarity
    return "Polarity: {}".format(polarity)

#endpoints - regression to predict house prices
@app.route('/pricing/', methods=['POST'])
@auth.required
def pricing():
    data = request.get_json()
    input_data = [data[col] for col in features]
    price = f" A house with a size of {input_data[0]}, built in the year of {input_data[1]} and with {input_data[2]} garage size has the price of {model.predict([input_data])[0]:.2f}"
    return price

To access the pricing and the sentiment functions, one will need to fill in the login and password. For the pricing function one will need the requests json and what else is needed before.

In [37]:
url = 'http://127.0.0.1:5000/pricing/'
auth= requests.auth.HTTPBasicAuth('someone', 'password')
response= requests.post(url, json=json_of_features, auth=auth)
response.text

' A house with a size of 120, built in the year of 2022 and with 2 garage size has the price of 207440.43'

# Final considerations

This ends the basics of machine learning deploy on APIs. From now one the API can be used with 2 different models, both using basic authentication. Things that could be added are: UI, more user capabilities and integration with other API to extract new data. 